# import

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy
from scipy.linalg import toeplitz
import scipy.ndimage.filters as filters
from scipy.io import savemat
from scipy import signal
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Conv2DTranspose
from tensorflow.keras.layers import Average, Add, Resizing, concatenate, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import LeakyReLU, PReLU, ThresholdedReLU
from tensorflow.keras import activations, regularizers

# parameters

In [ ]:
BATCH_SIZE = 64
ler        = 1E-3
epch       = 100
act        = 'elu'
act_end    = 'sigmoid'
s          = 'same'
reg        = regularizers.l1(l1=1e-5)

# functions

In [ ]:
@tf.function
def hfenn_loss(ori, res):
    fnorm = 0.325
    sigma = 1.5
    truncate = 4 
    wradius = int(truncate * sigma + 0.5)
    eye = np.zeros((2*wradius+1, 2*wradius+1), dtype=np.float32)
    eye[wradius, wradius] = 1.
    ker_mat = filters.gaussian_laplace(eye, sigma)
    with tf.name_scope('hfenn_loss'):
        chan = 3
        ker = tf.constant(np.tile(ker_mat[:, :, None, None], (1, 1, chan, 1)))
        filtered = tf.nn.depthwise_conv2d(ori - res, ker, [1, 1, 1, 1], 'VALID')
        loss = tf.reduce_mean(tf.square(filtered))
        loss = loss / (fnorm**2)
    return loss
  

def ae_loss(input_img, decoder):
    mse = tf.keras.losses.mean_squared_error(input_img, decoder) # MSE
    weight = 10.0 # weight
    return mse + weight * hfenn_loss(input_img, decoder) # MSE + weight * HFENN

In [ ]:
def datagen(img_folder, mask_folder):
    i = 0
    n = os.listdir(img_folder)
    
    while True:
        images = []
        masks  = []
        
        train_img_T  = sio.loadmat(img_folder +'/'+n[i])['temp'].ravel()
        train_msk_T  = sio.loadmat(mask_folder+'/'+n[i])['temp'].ravel()
        S_noisy      = FrST(train_img_T)
        S_clean      = FrST(train_msk_T)

        #noisy
        S_noisy_mag = np.absolute(S_noisy)
        images.append((S_noisy_mag - S_noisy_mag.min())/(S_noisy_mag.max() - S_noisy_mag.min()))

        #clean
        S_clean_mag = np.absolute(S_clean)
        S_clean_mag = (S_clean_mag - S_clean_mag.min())/(S_clean_mag.max() - S_clean_mag.min())
        masks.append(np.dstack((S_clean_mag, S_clean_mag, S_clean_mag)))

        i+=1
        if(i+1>=len(n)):
            i=0
        yield np.array(images),  np.array(masks)   
        
train_gen = datagen('C:/Users/Amir/denoising/newdataset256/train_noisy/train','C:/Users/Amir/denoising/newdataset256/train_clean/train')
val_gen   = datagen('C:/Users/Amir/denoising/newdataset256/val_noisy/val'    ,'C:/Users/Amir/denoising/newdataset256/val_clean/val'    )

# model structure

In [ ]:
def get_model(input_shape):
    
    ####################################### ENCODER ###############################################

    inputt = Input(shape=input_shape)
    x = Resizing(256, 512, interpolation="bilinear", crop_to_aspect_ratio=False)(inputt)
    
    x = Conv2D(  8, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D(  8, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D(  8, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    y0= BatchNormalization()(x)

    x = Conv2D( 16, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(y0)
    x = BatchNormalization()(x)
    x = Conv2D( 16, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    y1= BatchNormalization()(x)

    x = Conv2D( 32, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(y1)
    x = BatchNormalization()(x)
    x = Conv2D( 32, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    y2= BatchNormalization()(x)

    x = Conv2D( 64, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(y2)
    x = BatchNormalization()(x)
    x = Conv2D( 64, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    y3= BatchNormalization()(x)
    
    ##################################### BOTTLE NECK #############################################

    x = Conv2D(128, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(y3)
    x = BatchNormalization()(x)
    
    ####################################### DECODER ###############################################
    
    x = concatenate([y3,x])
    x = Conv2DTranspose( 64, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D( 64, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)

    x = concatenate([y2,x])
    x = Conv2DTranspose( 32, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D( 32, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)

    x = concatenate([y1,x])
    x = Conv2DTranspose( 16, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D( 16, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)

    x = concatenate([y0,x])
    x = Conv2DTranspose(  8, (3, 3), strides=(2,2), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D(  8, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Conv2D(  8, (3, 3), strides=(1,1), activation=act, padding=s, bias_regularizer=reg)(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(  1, (1,1), strides=(1,1), activation=act_end, padding=s)(x)
    endd = Resizing(128, 256, interpolation="bilinear", crop_to_aspect_ratio=False)(endd)
    tripleOut = concatenate([endd, endd, endd])
    
    return Model(inputs=inputt, outputs=tripleOut)


modell = get_model((128,256,1))
modell.summary()

# train

In [ ]:
n = len(os.listdir('C:/Users/Amir/denoising/newdataset256/train_noisy/train'))//BATCH_SIZE
m = len(os.listdir('C:/Users/Amir/denoising/newdataset256/val_noisy/val'))//BATCH_SIZE
weights_path = 'mag properties final'

opt    = tf.keras.optimizers.RMSprop(learning_rate=ler)
modell.compile(loss=ae_loss, optimizer=opt, metrics=['accuracy'])

checkpoint     = ModelCheckpoint(weights_path, monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
history        = modell.fit(train_gen, epochs=epch, steps_per_epoch = n, validation_data=val_gen, 
                            validation_steps=m, callbacks=[checkpoint], verbose=1)

In [ ]:
plt.plot(history.history['loss'][0:50])
plt.plot(history.history['val_loss'][0:50])
plt.title('model loss')   
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
ll = history.history['loss']
vl = history.history['val_loss']
np.save('ll.npy', ll)
np.save('vl.npy', vl)

# evaluation

In [ ]:
mytest1= np.zeros((1, 128, 256, 1)).astype('double')
mytest2= np.zeros((1, 128, 256, 1)).astype('double')

data   = sio.loadmat('C:/Users/Amir/denoising/result/0 dataset/test_noisy/test/ECG_629736.mat')['temp']
cleann = sio.loadmat('C:/Users/Amir/denoising/result/0 dataset/test_clean/test/ECG_629736.mat')['temp']

cleann  = cleann.T
cleann  = cleann.ravel()
S_clean = transform(cleann)

data    = data.T
data    = data.ravel()
S_noisy = transform(data)
S_noisy_mag = np.absolute(S_noisy)
S_noisy_phs = np.angle(S_noisy)

mytest1[0,:,:,0]= (S_noisy_mag - S_noisy_mag.min())/(S_noisy_mag.max() - S_noisy_mag.min())
testmodel          = get_model((128,256,1))
testmodel.load_weights('mag properties final')
evl             = testmodel.predict(mytest1)
cc = evl[0,:,:,0]*(S_noisy_mag.max() - S_noisy_mag.min()) + S_noisy_mag.min()

xx = cc * np.exp(1j * S_noisy_phs)

f, axarr = plt.subplots(3,2, figsize=(50,30))

axarr[0,0].plot(data)
axarr[0,1].imshow(S_noisy_mag)

ecg = itransform(xx)
axarr[1,0].plot(ecg)
axarr[1,1].imshow(cc)

axarr[2,0].plot(cleann)
axarr[2,1].imshow(np.absolute(S_clean))